In [24]:
import os
import openai
import pprint
openai.api_key = os.getenv("OPENAI_API_KEY")


In [25]:
from datasets import load_dataset

wine_dataset = load_dataset("alfredodeza/wine-ratings")

Found cached dataset csv (/Users/sinanozdemir/.cache/huggingface/datasets/alfredodeza___csv/alfredodeza--wine-ratings-7b131fffc73ec9d3/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/3 [00:00<?, ?it/s]

In [26]:
wine_dataset

DatasetDict({
    train: Dataset({
        features: ['name', 'region', 'variety', 'rating', 'notes'],
        num_rows: 32780
    })
    validation: Dataset({
        features: ['name', 'region', 'variety', 'rating', 'notes'],
        num_rows: 200
    })
    test: Dataset({
        features: ['name', 'region', 'variety', 'rating', 'notes'],
        num_rows: 200
    })
})

In [27]:
def convert_wine_to_string(wine):
    description = f'{wine["name"]} is from {wine["region"]} and is a {wine["variety"]}. {wine["notes"]}'
    
    return description

for wine in list(wine_dataset['test'])[:5]:
    print(convert_wine_to_string(wine))
    print()

Jim Barry Lodge Hill Shiraz 2008 is from Clare Valley, South Australia, Australia and is a Red Wine. This wine has deep, dark red/black hues, lifted aromas of mulberry, raspberry, coffee and kirsch. The bramble, chocolate, methol and rosemary fl avours that fi ll the mouth are present in abundance, with delicate, sensual and ripe tannins playing a structural role. This wine is ideally suited to good food, good friends and good conversation.

Argyle Reserve Pinot Noir 2005 is from Willamette Valley, Oregon and is a Red Wine. This Pinot Noir has a seductively rich, super dark ruby-violet color. Aromas leap out of the glass with black cherry, black cherry leather and cedar, along with a ripping portion of cinnamon spice. The wine finishes with a hint of violet flower patch. Use of new oak was delicate, so the aromas are authentic Pinot Noir.  

Cambria Katherine's Vineyard Chardonnay 1997 is from Central Coast, California and is a White Wine. The 1997 Katherines Vineyard Chardonnay is a r

In [28]:
import random

wines = random.sample(list(wine_dataset['test']), 5)

In [71]:
def pretty_print(data, indent=0):
    pp = pprint.PrettyPrinter(indent=indent)
    for item in data:
        for key, value in item.items():
            print(f"{key}: {value}")
        print()
    print()


In [86]:
# Define a dictionary for prices
prices = {
    'gpt-3.5-turbo': {
        'input': 0.0015 / 1000,  # per token
        'output': 0.002 / 1000  # per token
    }
}

def generate_openai_response(prompt, model='gpt-3.5-turbo', **kwargs):
    pretty_print(prompt)
    chat_completion = openai.ChatCompletion.create(
        model=model,
        messages=prompt,
        **kwargs
    )
    # Calculate the cost
    input_tokens = chat_completion['usage']['prompt_tokens']
    output_tokens = chat_completion['usage']['completion_tokens']
    input_cost = input_tokens * prices[model]['input']
    output_cost = output_tokens * prices[model]['output']
    total_cost = input_cost + output_cost
    return chat_completion, {'input_cost': input_cost, 'output_cost': output_cost, 'total_cost': total_cost}

In [87]:
user_description = 'This client hates red wines'

In [98]:
res, price = generate_openai_response(
[
    {'role': 'system', 'content': 'You are a wine bot that helps clients understand what kind of wine they want. Given a list of wines and a description of the client. Tell me what wines they want by giving me the  only the names of the wines. Include a reason preceding each pick to expalin to the user why they might like it.'}, 
    {'role': 'user', 'content': f'User: {user_description}\nWines:\n{f"{newline}".join([convert_wine_to_string(w) for w in wines])}'}
])
    
print(res.choices[0].message.content)



role: system
content: You are a wine bot that helps clients understand what kind of wine they want. Given a list of wines and a description of the client. Tell me what wines they want by giving me a numbered list of only the names of the wines. Include a reason preceding each pick to expalin to the user why they might like it.

role: user
content: User: This client hates red wines
Wines:
Chateau de Pez 2010 is from St. Estephe, Bordeaux, France and is a Red Wine. Chateau de Pez exhibits a deep color and harmonious composition. This complete, rich, dense wine merits prolonged aging. 

Carmel Limited Edition (OU Kosher) 2014 is from Israel and is a Red Wine. The wine displays a deep purplish hue and is characterized by a hint of ripe plum, cassis, violets mint and cigar boxes. Full-bodied and rich, with a long and balanced finish. The wine is ready to enjoy, but will continue to mature in the bottle in the coming years. The character of the wine reflects the "old wine" style of winemakin

In [100]:
price

{'input_cost': 0.0008265, 'output_cost': 0.000366, 'total_cost': 0.0011925}

In [102]:
40 / 0.0011925  # only 33k calls to hit 40 dollars now compared to hosting something on HuggingFace

33542.976939203356